In [ ]:
import numpy as np
import struct
import math
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import matplotlib as mpl
import pegasus_read as pegr
from matplotlib.pyplot import *
from scipy.ndimage import gaussian_filter
from get_kparofkprp import get_kparofkprp
import xarray

In [ ]:
name = 'b_b3_sim1'
nspecies = 1#7
plotsp = [1] #[0,2,1]
#charge = [1,1,1]
#mass = [1,1,1]
charge = [1,5,2]
mass = [1,16,4]


#output range  phi   ()
it0 = 50#25#0 #25 #65
it1 = 76#50#76 #50 #144
#output range f
itf0 = 146#73#146#73#0
itf1 = 222#146#222#222
#25 and 73, 50 and 146
#Try using a later range to match better (and use what constants Silvio has for each of his runs)
interv = "50-76/"
#interv = "25-50/"
#interv = "25-76/"

# window avg of f
rolling_window = 15#15

#cooling corrections
it0corr = 0
it1corr = 23 #25
cooling_corr = False # shouldnt affect SH here? check... #False

In [ ]:
# box parameters (beta = 0.3)
aspct = 6
lprp = 48.1802/(2.0*np.pi)                # in (2*pi*d_i) units   divide by 2pi
lprl = lprp*aspct         # in (2*pi*d_i) units
Lperp = 2.0*np.pi*lprp    # in d_i units
Lpara = 2.0*np.pi*lprl    # in d_i units
N_perp = 280  #ncells in perp direction
N_para = N_perp*aspct     # assuming isotropic resolution
kperpdi0 = 1./lprp        # minimum k_perp ( = 2*pi / Lperp)
kparadi0 = 1./lprl        # minimum k_para ( = 2*pi / Lpara)
betai0 = 0.3 #1./9.            # ion plasma beta
tau0 = 1.                 # temperature ratio (Te/Ti)
beta0 = (1.+tau0)*betai0  # total plasma beta

#number of processors used   (check if he computes D locally on each proc or not)
# seems the f read in is a sum over all processors (not local)
n_proc = 350*56

#k_perp shells
nkshells = 198#200

#binning type
bin_type = "linear"

In [ ]:
#gaussian filter
apply_smoothing = True #False
smooth_method = 'gaussian'
filter_passes = 1 #20 #10
#gaussian filter
sigma_smoothing = 0.5 #0.33
#Savitzky-Golay filter
window_size = 5
polyn_order = 3

In [ ]:
#numpy gradient: derivative order at edges
edge_grad_order = 2

#nomalization methods
same_norm_all = False #True #False

#exponential correction in diffusion coeff
exp_corr = True #False
c0exp = 0.09 #beta=0.3 0.05 #0.05

#shaded area: +/- c_pm_std*sigma (sigma = standard dev.)
c_pm_std = 1.0 #0.5

#--parameters for final plot  k_perp w_perp / \Omega = kappa  (translation from k_perp to w_perp for a given particle)
kappaU01 = 1.1 #beta = 0.3 1.25 #1.2
kappaB01 = 1.1 #1.25 #1.2
kappaPHI01 = 1.1 #1.25 #1.2

In [ ]:
#complement kpara*dBperp. dont know what this is
dBprp_complement = False #True #False
dtheta_dir = '1p5deg/'
path_strct_fnct = '../strct_fnct/'+dtheta_dir
m_order = '2'
cbprp_sign = +1.

#2*pi coefficient            k_perp lambda =1 or k_perp lambda = 2pi. Chose the former
TWOPIcoeff = False #True

#non-linear corrections
NLcorrections = False

In [ ]:
#figure format
output_figure = False #True #False
fig_frmt = ".pdf"#".png"#".pdf"
width_2columns = 512.11743/72.2
width_1column = 245.26653/72.2

#verbosity
verb_diag = False
verb_read = False

In [ ]:
#--rho_i units and KAW eigenvector normalization for density spectrum
kperprhoi0 = np.sqrt(betai0)*kperpdi0
kpararhoi0 = np.sqrt(betai0)*kparadi0
normKAW = betai0*(1.+betai0)*(1. + 1./(1. + 1./betai0))
#--
#--alfven speed (v_th units)
vA01 = np.sqrt(1./betai0)
#--d_i scale (rho_th units)
kdi01 = np.sqrt(betai0)

#--heating normalization
Qnormalization01 = 0.75 #0.4

In [ ]:
#paths
problem = "minor_turb"
path_save = "../figures/"
base = "../saved-analysis/spectrum_dat/"+name+"/"+name
#path_read_lev = "../fig_data/"

In [ ]:
# plot parameters

#latex fonts
font = 9
mpl.rc('text', usetex=True)
mpl.rc('font', family = 'serif')
mpl.rcParams['xtick.labelsize']=font-1
mpl.rcParams['ytick.labelsize']=font-1
#mpl.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]
mpl.rcParams['contour.negative_linestyle'] = 'solid'
plt.rcParams["font.weight"] = "normal"
plt.rcParams['xtick.top']=True
plt.rcParams['ytick.right']=True


#Hawley colormap
bit_rgb = np.linspace(0,1,256)
colors = [(0,0,127), (0,3,255), (0,255,255), (128,128,128), (255,255,0),(255,0,0),(135,0,0)]
positions = [0.0,0.166667,0.333333,0.5,0.666667,0.833333,1]
for iii in range(len(colors)):
 colors[iii] = (bit_rgb[colors[iii][0]],
                bit_rgb[colors[iii][1]],
                bit_rgb[colors[iii][2]])

cdict = {'red':[], 'green':[], 'blue':[]}
for pos, color in zip(positions, colors):
 cdict['red'].append((pos, color[0], color[0]))
 cdict['green'].append((pos, color[1], color[1]))
 cdict['blue'].append((pos, color[2], color[2]))

cmap = mpl.colors.LinearSegmentedColormap('my_colormap',cdict,256)

In [ ]:
print("\n [ reading initial condition ]")
vdf0, time, vprp0, vprl0 = pegr.readmat_vdf(name,0,1,nspecies,plotsp,edv=False,grid=True,verbose=True)

#first normalization by number of processors
for i in np.arange(nspecies):
    vdf0[i] = vdf0[i][0,:,:] / float(n_proc)

In [ ]:
if cooling_corr:
    #correcting for numerical cooling
    print("\n [ calculate cooling correction (vs w_perp) ]")
    # if not apply_smoothing - store in edotv_prp_smooth arrays etc instead
    # can just do everything in one line instead
    # calculate here to not overload storage once main vdfs are read in
    blank1, blank2, edotv_prp_, edotv_prl_, blank3, blank4 = pegr.readmat_vdf(name,it0corr,it1corr+1,nspecies,plotsp,edv=True,grid=True,verbose=verb_read)

    for sp in np.arange(nspecies):
        edotv_prl_[sp] = edotv_prl_[sp] / float(n_proc)
        edotv_prp_[sp] = edotv_prp_[sp] / float(n_proc)

        if apply_smoothing:
            for ifilt in range(filter_passes):
              edotv_prp_[sp] = gaussian_filter(edotv_prp_[sp],sigma=sigma_smoothing,axes=(1,2))
              edotv_prl_[sp] = gaussian_filter(edotv_prl_[sp],sigma=sigma_smoothing,axes=(1,2))

    edotv_prl_corr = []
    edotv_prp_corr = []
    edotv_prl_smooth_corr = []
    edotv_prp_smooth_corr = []

    for sp in np.arange(nspecies):
        edotv_prl_corr.append(np.mean(edotv_prl_[sp],axis=0,keepdims=True))
        edotv_prp_corr.append(np.mean(edotv_prp_[sp],axis=0,keepdims=True))
        if (not apply_smoothing):
            edotv_prl_smooth_corr.append(gaussian_filter(edotv_prl_[sp],sigma=sigma_smoothing,axes=(1,2)))
            edotv_prp_smooth_corr.append(gaussian_filter(edotv_prp_[sp],sigma=sigma_smoothing,axes=(1,2)))
            for ifilt in range(filter_passes-1):
                edotv_prl_smooth_corr[sp] = gaussian_filter(edotv_prl_smooth_corr[sp],sigma=sigma_smoothing,axes=(1,2))
                edotv_prp_smooth_corr[sp] = gaussian_filter(edotv_prp_smooth_corr[sp],sigma=sigma_smoothing,axes=(1,2))
            edotv_prl_smooth_corr[sp] = np.mean(edotv_prl_smooth_corr[sp],axis=0,keepdims=True)
            edotv_prp_smooth_corr[sp] = np.mean(edotv_prp_smooth_corr[sp],axis=0,keepdims=True)

    del blank1, blank2, edotv_prp_, edotv_prl_, blank3, blank4

In [ ]:
### HEATING VS W_PERP (beta = 0.3)
#
# -> reading simulation data, time averaging, and cooling corrections
# -> also: reading spectra of fluctuations, reducing to k_perp spectra, and time averaging
#
print("\n ### HEATING VS W_PERP ###")

# read in VDF, edotv file saved using jono's scripts
vdf_, time, edotv_prp_, edotv_prl_, vprp, vprl = pegr.readmat_vdf(name,itf0,itf1+1,nspecies,plotsp,edv=True,grid=True,verbose=verb_read)

time = time[itf0:itf1+1,:]

vdf_avg = []
edotv_prp_t = []
edotv_prl_t = []
edotv_prp_avg = []
edotv_prl_avg = []
dfdwprp_t = []
dfdwprp_avg = []

for sp in np.arange(nspecies):
  if verb_diag:
    print("\n")
    print("#########################################################")
    print("### v-space analysis: distribution function & heating ###")
    print("#########################################################")
  print("species",plotsp[sp])

  # is assumed dvprp and dvprl are the same for all species later on. (can change that)
  dvprp = vprp[sp][0,2,0]-vprp[sp][0,1,0]
  dvprl = vprl[sp][0,0,2]-vprl[sp][0,0,1]

  #first normalization by number of processors
  vdf_[sp] = vdf_[sp] / float(n_proc)
  edotv_prl_[sp] = edotv_prl_[sp] / float(n_proc)
  edotv_prp_[sp] = edotv_prp_[sp] / float(n_proc)

  if apply_smoothing:
    for ifilt in range(filter_passes):
      edotv_prp_[sp] = gaussian_filter(edotv_prp_[sp],sigma=sigma_smoothing,axes=(1,2))
      edotv_prl_[sp] = gaussian_filter(edotv_prl_[sp],sigma=sigma_smoothing,axes=(1,2))

  # average over a rolling window (could also try fixed bin avgs)
  vdf_avg.append(xarray.DataArray(vdf_[sp],dims=['t','wprp','wprl']).rolling(t=rolling_window,min_periods=1, center=True).mean().to_numpy())
  edotv_prp_avg.append(xarray.DataArray(edotv_prp_[sp],dims=['t','wprp','wprl']).rolling(t=rolling_window,min_periods=1, center=True).mean().to_numpy())
  edotv_prl_avg.append(xarray.DataArray(edotv_prl_[sp],dims=['t','wprp','wprl']).rolling(t=rolling_window,min_periods=1, center=True).mean().to_numpy())

  edotv_prp_t.append(edotv_prp_[sp]*1.0)
  edotv_prl_t.append(edotv_prl_[sp]*1.0)

  #computing <df/dwprp>
  fvprp_t = np.sum(vdf_[sp]/vprp[sp]*dvprl,axis=2) / np.sum(np.sum(vdf_[sp]/vprp[sp]*dvprl*dvprl,axis=2),axis=1,keepdims=True)
  dfdwprp_t.append(np.gradient(fvprp_t,vprp[sp][0,:,0],edge_order=edge_grad_order,axis=1))
  dfdwprp_avg.append(xarray.DataArray(dfdwprp_t[sp],dims=['t','wprp']).rolling(t=rolling_window,min_periods=1, center=True).mean().to_numpy())

In [ ]:
np.gradient(fvprp_t,vprp[sp][0,:,0],edge_order=edge_grad_order,axis=1)

In [ ]:
vprp[sp][:,:,0]

In [ ]:
  ### reading spectra of fluctuations
  ### NOTE: these are all in code units [i.e., both kprp and E(kprp) ]

for ind in range(it0,it1+1):
    filename1 = base+"."+"%05d"%ind+".spectrum1d.nkperp"+"%d"%nkshells+"."+bin_type+".Uprp.dat"
    filename2 = base+"."+"%05d"%ind+".spectrum1d.nkperp"+"%d"%nkshells+"."+bin_type+".Bprl.dat"
    filename3 = base+"."+"%05d"%ind+".spectrum1d.nkperp"+"%d"%nkshells+"."+bin_type+".PHI.dat"
    if dBprp_complement:
        filename4 = base+"."+"%05d"%ind+".spectrum1d.nkperp"+"%d"%nkshells+"."+bin_type+".Bprp.dat"
        filename5 = base+"."+"%05d"%ind+".spectrum1d.nkperp"+"%d"%nkshells+"."+bin_type+".B.dat"
    if verb_diag:
        print("\n  [reading spectrum of the fluctuations]")
        print("    ->",filename1)
        print("    ->",filename2)
        print("    ->",filename3)
    if dBprp_complement:
        print("    ->",filename4)

    #Phi spectrum vs k_perp - probably just what we want
    dataPhikprp = np.loadtxt(filename3)

    if (ind == it0):
        #generating 1D arrays for the first time
        kprp_phi = dataPhikprp[1:,0]
        Phikprp = np.zeros(len(kprp_phi))
        Phikprp_t = np.zeros([len(kprp_phi),it1-it0+1])

    # average over time window
    Phikprp[:] = Phikprp[:] + dataPhikprp[1:,1]/float(it1-it0+1)

    Phikprp_t[:,ind-it0] = dataPhikprp[1:,1]

    # end loop over time

# interpolate to time cadence of VDF data - but VDF more frequent...
#phi_t_arr = np.arange(77)*28.90811
#for ind in range(Phikprp_t.shape[0]):
#  Phikprp_t[ind,:] = np.interp(time[:,0] phi_t_arr, Phikprp_t[ind,:])

kprp = kprp_phi
kprp_phi_rho = np.sqrt(betai0)*kprp_phi

In [ ]:
phi_t_arr = np.arange(it0,it1+1)*28.90811
dfdwprp_t_interp = []
for sp in np.arange(nspecies):
    dfdwprp_t_interp.append(np.zeros((it1+1-it0,dfdwprp_t[sp].shape[1])))
    for ind in range(dfdwprp_t[sp].shape[1]):
        dfdwprp_t_interp[sp][:,ind] = np.interp(phi_t_arr, time[:,0], dfdwprp_t[sp][:,ind])

In [ ]:
#vdf_[sp]/vprp[sp]
#vdf output is actually vperp*f: restoring f
# check if jono's script already corrects for this
vdf = []
for sp in np.arange(nspecies):
    vdf.append(vdf_avg[sp] / vprp[sp])
    vdf0[sp] = vdf0[sp] / vprp0[sp][0,:,:]

edotv_prl = edotv_prl_avg
edotv_prp = edotv_prp_avg
if (not apply_smoothing):
  edotv_prl_smooth = edotv_prl_smooth_avg
  edotv_prp_smooth = edotv_prp_smooth_avg

In [ ]:
print("\n")
print("########################## BETA = 0.3 ##########################")
print(" (from v space -- before cooling corrections) ")
print(" after 3.5t_A")
for sp in  np.arange(nspecies):
    print(" species ",plotsp[sp])
    print(" 1) integral of <Qperp> (code units): ",np.mean(np.sum(edotv_prp[sp][:,:,:]*dvprp*dvprl,axis=(1,2))))
    print(" 2) integral of <Qpar> (code units): ",np.mean(np.sum(edotv_prl[sp][:,:,:]*dvprp*dvprl,axis=(1,2))))
print("################################################################")
print("\n")

#computing d<f>/dw_perp
#dfdwprp = np.gradient(np.sum(vdf*dvprl,axis=1),vprp)
# f_vprp using rolling averaged f, and then calculating dfdwprp
f_vprp = []
f0_vprp = []
dfdwprp = []
dfdwprp_finitediff = []
gg_tmp = []
gg_tmp_smooth = []

for sp in np.arange(nspecies):
    f_vprp.append(np.sum(vdf[sp]*dvprl,axis=2)/np.abs(  np.sum(np.sum(vdf[sp]*dvprl*dvprp,axis=2),axis=1,keepdims=True)   ))
    f0_vprp.append(np.sum(vdf0[sp]*dvprl,axis=1)/np.abs(np.sum(vdf0[sp]*dvprl*dvprp)))
    dfdwprp.append(np.gradient(f_vprp[sp],vprp[sp][0,:,0],edge_order=edge_grad_order,axis=1))
    dfdwprp_finitediff.append(np.zeros(f_vprp[sp].shape))
    dfdwprp_finitediff[sp][:,1:-1] = (f_vprp[sp][:,2:]-f_vprp[sp][:,:-2])/(vprp[sp][:,2:,0]-vprp[sp][:,:-2,0])
    dfdwprp_finitediff[sp][:,0] = (f_vprp[sp][:,1]-f_vprp[sp][:,0])/(vprp[sp][:,1,0]-vprp[sp][:,0,0])
    dfdwprp_finitediff[sp][:,len(vprp[sp][0,:,0])-1] = (f_vprp[sp][:,len(vprp[sp][0,:,0])-1]-f_vprp[sp][:,len(vprp[sp][0,:,0])-2])/(vprp[sp][:,len(vprp[sp][0,:,0])-1,0]-vprp[sp][:,len(vprp[sp][0,:,0])-2,0])

# rolling avg edotv
    gg_tmp.append(edotv_prp[sp]/(np.abs( np.abs(np.sum(edotv_prl[sp]*dvprp*dvprl,axis=(1,2),keepdims=True)) + np.abs(np.sum(edotv_prp[sp]*dvprp*dvprl,axis=(1,2),keepdims=True)) )))
    edotv_prp_t[sp] /= (np.abs( np.abs(np.sum(edotv_prl_t[sp]*dvprp*dvprl,axis=(1,2),keepdims=True)) + np.abs(np.sum(edotv_prp_t[sp]*dvprp*dvprl,axis=(1,2),keepdims=True)) ))
    if (not apply_smoothing):
        gg_tmp_smooth.append(edotv_prp_smooth[sp]/(np.abs( np.abs(np.sum(edotv_prl_smooth[sp]*dvprp*dvprl,axis=(1,2),keepdims=True)) + np.abs(np.sum(edotv_prp_smooth[sp]*dvprp*dvprl,axis=(1,2),keepdims=True)) )))
        edotv_prp_smooth_t[sp] /= (np.abs( np.abs(np.sum(edotv_prl_smooth_t[sp]*dvprp*dvprl,axis=(1,2),keepdims=True)) + np.abs(np.sum(edotv_prp_smooth_t[sp]*dvprp*dvprl,axis=(1,2),keepdims=True)) ))

In [ ]:
if cooling_corr:
    #correcting for numerical cooling
    print("\n [ apply cooling correction (vs w_perp) ]")
    """
    # if not apply_smoothing - store in edotv_prp_smooth arrays etc instead
    # can just do everything in one line instead
    edotv_prl_corr.append(np.mean(edotv_prl_[sp][it0corr:it1corr+1,:,:],axis=0,keepdims=True))
    edotv_prp_corr.append(np.mean(edotv_prp_[sp][it0corr:it1corr+1,:,:],axis=0,keepdims=True))
    if (not apply_smoothing):
        edotv_prl_smooth_corr.append(gaussian_filter(edotv_prl_[sp][it0corr:it1corr+1,:,:],sigma=sigma_smoothing,axes=(1,2)))
        edotv_prp_smooth_corr.append(gaussian_filter(edotv_prp_[sp][it0corr:it1corr+1,:,:],sigma=sigma_smoothing,axes=(1,2)))
        for ifilt in range(filter_passes-1):
            edotv_prl_smooth_corr[sp] = gaussian_filter(edotv_prl_smooth_corr[sp],sigma=sigma_smoothing,axes=(1,2))
            edotv_prp_smooth_corr[sp] = gaussian_filter(edotv_prp_smooth_corr[sp],sigma=sigma_smoothing,axes=(1,2))
        edotv_prl_smooth_corr[sp] = np.mean(edotv_prl_smooth_corr[sp],axis=0,keepdims=True)
        edotv_prp_smooth_corr[sp] = np.mean(edotv_prp_smooth_corr[sp],axis=0,keepdims=True)
    """

    gg_tmp[sp] -= edotv_prp_corr[sp]/(np.abs( np.abs(np.sum(edotv_prl_corr[sp]*dvprp*dvprl,keepdims=True)) + np.abs(np.sum(edotv_prp_corr[sp]*dvprp*dvprl,keepdims=True)) ))
    edotv_prp_t[sp] -= edotv_prp_corr[sp]/(np.abs( np.abs(np.sum(edotv_prl_corr[sp]*dvprp*dvprl,keepdims=True)) + np.abs(np.sum(edotv_prp_corr[sp]*dvprp*dvprl,keepdims=True)) ))

    if (not apply_smoothing):
        gg_tmp_smooth[sp] -= edotv_prp_smooth_corr[sp]/(np.abs( np.abs(np.sum(edotv_prl_smooth_corr[sp]*dvprp*dvprl,keepdims=True)) + np.abs(np.sum(edotv_prp_smooth_corr[sp]*dvprp*dvprl,keepdims=True)) ))
        edotv_prp_smooth_t[sp] -= edotv_prp_smooth_corr[sp]/(np.abs( np.abs(np.sum(edotv_prl_smooth_corr[sp]*dvprp*dvprl,keepdims=True)) + np.abs(np.sum(edotv_prp_smooth_corr[sp]*dvprp*dvprl,keepdims=True)) ))

In [ ]:
np.sum(gg_tmp[sp])#,axis=2)

In [ ]:
np.nonzero(edotv_prp_[sp])

In [ ]:
#edotv_prp[sp][200]

In [ ]:
np.sum(edotv_prp_t[sp]*dvprl,axis=2)

In [ ]:
Qprp_vprp = []
Qprp_vprp_t = []
Qprp_vprp_t_avg = []
Qprp_vprp_smooth = []
Qprp_vprp_smooth_t = []
Qprp_vprp_smooth_t_avg = []
stdQprp_vprp = []
stdQprp_vprp_smooth = []
Dprpprp = []
Dprpprp_sign = []
Dprpprp_t = []
Dprpprp_t_avg = []
Dprpprp_smooth = []
Dprpprp_sign_smooth = []
Dprpprp_smooth_t = []
Dprpprp_smooth_t_avg = []
stdDprpprp_vprp = []
stdDprpprp_vprp_smooth = []

for sp in np.arange(nspecies):
    # total perpendicular heating from edotv
    # does this need to be normalised by qom? since E field acceleration
    Qprp_vprp.append(np.sum(gg_tmp[sp]*dvprl,axis=2))
    Qprp_vprp_t.append(np.sum(edotv_prp_t[sp]*dvprl,axis=2))
    Qprp_vprp_t_avg.append(xarray.DataArray(Qprp_vprp_t[sp],dims=['t','wprp']).rolling(t=rolling_window,min_periods=1, center=True).mean().to_numpy())
    if (not apply_smoothing):
      Qprp_vprp_smooth.append(np.sum(gg_tmp_smooth[sp]*dvprl,axis=2))
      Qprp_vprp_smooth_t.append(np.sum(edotv_prp_smooth_t[sp]*dvprl,axis=2))
      Qprp_vprp_smooth_t_avg.append(xarray.DataArray(Qprp_vprp_smooth_t[sp],dims=['t','wprp']).rolling(t=rolling_window,min_periods=1, center=True).mean().to_numpy())

    #normalize Qprp_sim (after or before computing Dprpprp?)
    #normQ_sim[sp] = Qnormalization01/np.sum(np.abs(Qprp_vprp[sp])*dvprp,axis=1,keepdims=True)
    #print("  -> denom of normQ_sim = ",np.sum(np.abs(Qprp_vprp[sp])*dvprp,axis=1))
    Qprp_vprp[sp] *= Qnormalization01/np.sum(np.abs(Qprp_vprp[sp])*dvprp,axis=1,keepdims=True)
    #Qprp_vprp_t *= normQ_sim
    Qprp_vprp_t[sp] *= Qnormalization01/np.sum(np.abs(Qprp_vprp_t[sp])*dvprp,axis=1,keepdims=True)
    #for iit in range(it0,it1+1):
    #  Qprp_vprp_t[:,iit-it0] *= Qnormalization01/np.sum(np.abs(Qprp_vprp_t[:,iit-it0])*dvprp)
    #Qprp_vprp_t_avg *= normQ_sim
    Qprp_vprp_t_avg[sp] *= Qnormalization01/np.sum(np.abs(Qprp_vprp_t_avg[sp])*dvprp,axis=1,keepdims=True)

    #std in time
    stdQprp_vprp.append(np.std(Qprp_vprp_t[sp],axis=0))

    if (not apply_smoothing):
        Qprp_vprp_smooth[sp] *= Qnormalization01/np.sum(np.abs(Qprp_vprp_smooth[sp])*dvprp,axis=1,keepdims=True)
        Qprp_vprp_smooth_t[sp] *= Qnormalization01/np.sum(np.abs(Qprp_vprp_smooth_t[sp])*dvprp,axis=1,keepdims=True)
        Qprp_vprp_smooth_t_avg[sp] *= Qnormalization01/np.sum(np.abs(Qprp_vprp_smooth_t_avg[sp])*dvprp,axis=1,keepdims=True)
        stdQprp_vprp_smooth.append(np.std(Qprp_vprp_smooth_t[sp],axis=0))


    #computing diff coefficient  heating/dfdw
    #Get rid of divisions by zero that we get because of how many wprp we go out to (where there are no particles/f)
    #Q is mostly small in these regions...
    #dfdw0s = np.ones(dfdwprp[sp].shape)-np.isnan(1./dfdwprp[sp])
    #dfdw0s_t = np.ones(dfdwprp_t[sp].shape)-np.isnan(1./dfdwprp_t[sp])
    Dprpprp.append(- np.abs(Qprp_vprp[sp]) / dfdwprp[sp])
    Dprpprp[sp][np.isinf(1./dfdwprp[sp])]=0
    Dprpprp_sign.append(- Qprp_vprp[sp] / dfdwprp[sp])
    Dprpprp_sign[sp][np.isinf(1./dfdwprp[sp])]=0
    Dprpprp_t.append(- np.abs(Qprp_vprp_t[sp]) / dfdwprp_t[sp])
    Dprpprp_t[sp][np.isinf(1./dfdwprp_t[sp])]=0
    Dprpprp_t_avg.append(xarray.DataArray(Dprpprp_t[sp],dims=['t','wprp']).rolling(t=rolling_window,min_periods=1, center=True).mean().to_numpy())
    #Dprpprp_t_avg[sp][np.isnan(Dprpprp_t_avg[sp])]=0
    stdDprpprp_vprp.append(np.std(Dprpprp_t[sp],axis=0))

    if (not apply_smoothing):
      Dprpprp_smooth.append(- np.abs(Qprp_vprp_smooth[sp]) / dfdwprp[sp])
      Dprpprp_smooth[sp][np.isinf(1./dfdwprp[sp])]=0
      Dprpprp_sign_smooth.append(- Qprp_vprp_smooth[sp] / dfdwprp[sp])
      Dprpprp_sign_smooth[sp][np.isinf(1./dfdwprp[sp])]=0
      Dprpprp_smooth_t.append(- np.abs(Qprp_vprp_smooth_t[sp]) / dfdwprp_t[sp])
      Dprpprp_smooth_t[sp][np.isinf(1./dfdwprp_t[sp])]=0
      Dprpprp_smooth_t_avg.append(xarray.DataArray(Dprpprp_smooth_t[sp],dims=['t','wprp']).rolling(t=rolling_window,min_periods=1, center=True).mean().to_numpy())
      #Dprpprp_smooth_t_avg[sp][np.isnan(Dprpprp_smooth_t_avg[sp])]=0
      stdDprpprp_vprp_smooth.append(np.std(Dprpprp_smooth_t[sp],axis=0))

    print("species ",sp)
    print("\n ##### CHECK: integral of simulation curves (beta_i = 0.3) ##### (avg after 3.5t_A)")
    print("  -> normQ_sim = ",Qnormalization01/np.mean(np.sum(np.abs(Qprp_vprp[sp])*dvprp,axis=1)[100:]))
    print("  -> integral of df/dwprp..")
    print("     i) ..from np.gradient:",np.mean(np.sum(dfdwprp[sp]*dvprp,axis=1)[100:]))
    print("    ii) ..from finite diff:",np.mean(np.sum(dfdwprp_finitediff[sp]*dvprp,axis=1)[100:]))
    print("    ..to be compared with f(vprp=0):",np.mean(f_vprp[sp][100:,0]))
    #print("   [dfdwprp - dfdwprp_finitediff =",dfdwprp - dfdwprp_finitediff,"]")
    print("  -> integral of dQprp/dwprp: ",np.mean(np.sum(Qprp_vprp[sp]*dvprp,axis=1)[100:]))
    print("  -> integral of Dprpprp.. ")
    print("     i) ..from < dQ/dwprp > / < df/dwprp > :",np.mean(np.sum(Dprpprp[sp]*dvprp,axis=1)[100:]))
    print("    ii) ..from < (dQ/dwprp) / (df/dwprp) > :",np.mean(np.sum(Dprpprp_t_avg[sp]*dvprp,axis=1)[100:]))
    print(" ################################################################\n")

In [ ]:
np.sum(np.isnan(1./dfdwprp[0]))

In [ ]:
np.sum(np.isinf(1./dfdwprp[0]))

In [ ]:
np.sum(np.isinf(Qprp_vprp[0]))

In [ ]:
np.sum(np.isinf(Qprp_vprp[0]/dfdwprp[0]))

In [ ]:
np.sum(np.isnan(Qprp_vprp[0]/dfdwprp[0]))

In [ ]:
Qprp_vprp[0][np.isinf(Qprp_vprp[0]/dfdwprp[0])]

In [ ]:
Qprp_vprp[0][np.isnan(Qprp_vprp[0]/dfdwprp[0])]

In [ ]:
### COMPUTING THEORETICAL CURVES (Dprpprp and Qprp) FROM FLUCTUATIONS
#
# beta = 0.3
alphaU = kappaU01
alphaB = kappaB01
alphaPHI = kappaPHI01

In [ ]:
#np.interp(vprp[sp][0,:,0], alphaPHI/kprp_phi_rho[::-1], dPhi_k[::-1])

In [ ]:
1.0/alphaPHI

In [ ]:
charge[plotsp[0]]/np.sqrt(mass[plotsp[0]])

In [ ]:
c0exp = 0.06 #beta=0.3 0.05 #0.05
alphaPHI = 1.1#1.1 #1.25 #1.2
cst01 = 0.9 #0.9   #1.05

In [ ]:
dPhi_v = [] #Phi vs vprp - different species may have different vprp scales
dPhi_v_t = []
Dprpprp_Phik = []
Dprpprp_Phik_t = []
Dprpprp_Phik_t_avg = []
Qprp_Phik = []
Qprp_Phik_t = []
Qprp_Phik_t_avg = []
normD_phi01 = []
normD_phi01_t_avg = []
Qprp_k_phik = []
Qprp_k_phik_t_avg = []
for sp in np.arange(nspecies):
    # calculate k phi/beta and interpolate and convert to v-space
    dPhi_k = np.sqrt(kprp_phi_rho*Phikprp/np.sqrt(betai0)) # attention! spectra were originally computed in kprp*di units!
    #dPhi_v.append(np.interp(vprp[sp][0,:,0], alphaPHI/kprp_phi_rho[::-1], dPhi_k[::-1]))
    dPhi_v.append(np.interp(vprp[sp][0,:,0], charge[plotsp[sp]]/np.sqrt(mass[plotsp[sp]]) * alphaPHI/kprp_phi_rho[::-1], dPhi_k[::-1]))
    dPhi_v[sp] /= betai0 #thermal units
    #
    dPhi_k_t = np.zeros([len(kprp_phi_rho),it1-it0+1])
    dPhi_v_t.append(np.zeros([len(vprp[sp][0,:,0]),it1-it0+1]))
    for ind in range(it0,it1+1):
      dPhi_k_t[:,ind-it0] = np.sqrt(kprp_phi_rho[:]*Phikprp_t[:,ind-it0]/np.sqrt(betai0)) # attention! spectra were originally computed in kprp*di units!
      #dPhi_v_t[sp][:,ind-it0] = np.interp(vprp[sp][0,:,0], alphaPHI/kprp_phi_rho[::-1], dPhi_k_t[::-1,ind-it0])
      dPhi_v_t[sp][:,ind-it0] = np.interp(vprp[sp][0,:,0], charge[plotsp[sp]]/np.sqrt(mass[plotsp[sp]]) * alphaPHI/kprp_phi_rho[::-1], dPhi_k_t[::-1,ind-it0])
      dPhi_v_t[sp][:,ind-it0] = dPhi_v_t[sp][:,ind-it0] / betai0

    # multiply the potential by the charge of the species of interest
    dPhi_v[sp] *= charge[plotsp[sp]]
    dPhi_v_t[sp] *= charge[plotsp[sp]]
    #
    #--second: corresponding diffusion coefficients
    #
    Dprpprp_Phik.append((dPhi_v[sp]**3.) / (vprp[sp][0,:,0]**2.) )           #diffusion coefficient associated to Phi_tot (in w_perp/v_th)
    # We want dPhi_v - the fully combined phi
    # Dprpprp_Phik calculates using time-averaged phi
    #
    Dprpprp_Phik_t.append(np.zeros([len(vprp[sp][0,:,0]),it1-it0+1]) )
    for ind in range(it0,it1+1):
      Dprpprp_Phik_t[sp][:,ind-it0] = (dPhi_v_t[sp][:,ind-it0]**3.) / (vprp[sp][0,:,0]**2.)
      if exp_corr:
         Dprpprp_Phik_t[sp][:,ind-it0] = Dprpprp_Phik_t[sp][:,ind-it0]*np.exp(-c0exp*(vprp[sp][0,:,0]**2./dPhi_v_t[sp][:,ind-it0]))
    Dprpprp_Phik_t[sp] *= charge[plotsp[sp]] / mass[plotsp[sp]]
    Dprpprp_Phik_t_avg.append(np.sum(Dprpprp_Phik_t[sp],axis=1)/float(it1-it0+1) )
    #
    # Dprpprp_Phik_t_avg calculates time dependent Dprpprp, and takes a time average
    if exp_corr:
       Dprpprp_Phik[sp] *= np.exp(-c0exp*(vprp[sp][0,:,0]**2./dPhi_v[sp]))           #apply exponential correction (but using: Phi_comb or Phi_tot)

    Dprpprp_Phik[sp] *= charge[plotsp[sp]] / mass[plotsp[sp]]
    # Matt's boardwork missed a factor of m in the definition of D (Eq.7 in Silvio's paper)

    #
    #--third: corresponding differential heating
    #
    Qprp_Phik.append(- Dprpprp_Phik[sp].reshape(1,vprp[sp].shape[1]) * dfdwprp[sp] )           #differential perpendicular heating associated to Phi_tot (in w_perp/v_th)
    #^ has time as first dimension instead of second
    Qprp_Phik_t.append(np.zeros([len(vprp[sp][0,:,0]),it1-it0+1]))
    for ind in range(it0,it1+1):
      Qprp_Phik_t[sp][:,ind-it0] = - Dprpprp_Phik_t[sp][:,ind-it0] * dfdwprp_t_interp[sp][ind-it0,:]
    Qprp_Phik_t_avg.append(np.sum(Qprp_Phik_t[sp],axis=1)/float(it1-it0+1))

    ### normalizations (made in w_perp/v_th space, then ported to k_perp*rho_th space)
    #
    #--Dprpprp vs wprp
    #
    #cst01 = 0.9 #1.0
    #
    i01 = np.where(vprp[sp][0,:,0] > 1.0 / (charge[plotsp[sp]]/np.sqrt(mass[plotsp[sp]]) * alphaPHI))[0][0]  #--match Dprpprp_phi with Dprpprp_sim (0.6?)  #1.0/alphaPHI
    #
    #normalisations to make things line up through a single w_perp - we don't care about the individual components

    if apply_smoothing:
      normD_phi01.append(cst01*Dprpprp[sp][:,i01] / Dprpprp_Phik[sp][i01] )#one normalisation for all t, or each t - plot this vs t to see if it changes much in time, taking the mean for now
      normD_phi01_t_avg.append(cst01*Dprpprp[sp][:,i01] / Dprpprp_Phik_t_avg[sp][i01] )
    else:
      normD_phi01 = cst01*Dprpprp_smooth[sp][:,i01] / Dprpprp_Phik[sp][i01]
      normD_phi01_t_avg = cst01*Dprpprp_smooth[sp][:,i01] / Dprpprp_Phik_t_avg[sp][i01]

    # set minor ion normalisations to the same as for protons
    #if plotsp[sp]!=0:
    #  normD_phi01[sp] = 32728410.812698353
    #  normD_phi01_t_avg[sp] = 31506759.908403452

    #
    Dprpprp_Phik[sp] *= np.mean(normD_phi01[sp])
    Dprpprp_Phik_t_avg[sp] *= np.mean(normD_phi01_t_avg[sp])
    #
    print('\n')
    print('### betai0 = 0.3 ###')
    print('species',sp)
    print('* kappa0 values: ')
    print('  effective kappa0_PHI = ',alphaPHI)
    print('* D_perpperp normalizations: ')
    print('  normD_Phi =',np.mean(normD_phi01[sp]))
    print('  normD_Phi_t_avg =',np.mean(normD_phi01_t_avg[sp]))
    #
    #--Qprp vs wprp (apply same normalization as for Dprpprp)
    Qprp_Phik[sp] *= np.mean(normD_phi01[sp])
    Qprp_Phik_t_avg[sp] *= np.mean(normD_phi01_t_avg[sp])
    #
    #####
    #
    # << vs k_perp >>
    #
    # -> just interpolate dQperp/dwperp into k_perp*rho_th
    #    (plus extra contribution due to the fact that we
    #     need to compute dQperp/dlog(k_perp) and not dQperp/dk_perp)
    #    [ normalization should already be consistent! ]
    #
    #--heating in k_perp from Phi spectrum
    Qprp_k_phik.append(np.zeros((Qprp_Phik[sp].shape[0],kprp_phi_rho.shape[0])))

    for t in np.arange(Qprp_Phik[sp].shape[0]):
        Qprp_k_phik[sp][t] = np.interp( kprp_phi_rho, alphaPHI/vprp[sp][0,::-1,0], Qprp_Phik[sp][t,::-1] )
    Qprp_k_phik[sp] *= (alphaPHI/kprp_phi_rho.reshape((1,kprp_phi_rho.shape[0])))

    Qprp_k_phik_t_avg.append(np.interp( kprp_phi_rho, alphaPHI/vprp[sp][0,::-1,0], Qprp_Phik_t_avg[sp][::-1] ))
    Qprp_k_phik_t_avg[sp] *= (alphaPHI/kprp_phi_rho)
    Qprp_k_phik_t_avg[sp] /= np.log(10.)

In [ ]:
#--set ranges
#
# vs w_perp
xr_min_w = 0.1
xr_max_w = []
yr_max_f_w = []
yr_max_Q_w = []
for sp in np.arange(nspecies):
    xr_max_w.append(np.max(vprp[sp]) )
    yr_max_f_w.append(1.025*np.max([np.max(f_vprp[sp]),np.max(f0_vprp[sp])]) )
    yr_max_Q_w.append(1.1*np.max([np.max(Qprp_vprp[sp])]))
xr_max_w = np.max(np.array(xr_max_w))#[plotsp]))
yr_max_f_w = np.max(np.array(yr_max_f_w))#[plotsp]))
yr_max_Q_w = np.max(np.array(xr_max_w))#[plotsp]))
yr_min_f_w = 0.0
yr_min_Q_w = -0.05 #1.05*np.min([np.min(Qprp_vprp),np.min(Qprp_vprp03)])
yr_min_D_w = 0.9e-1 #8e-3#8e-1 #np.min(np.abs(Dprpprp_corrected))
yr_max_D_w = 1.5e+1 #8e+1#8e+3 #np.max(np.abs(Dprpprp_corrected))
#


#--lines and fonts
line_thick = 1.0 #1.25
line_thick_aux = 0.75
lnstyl = ['-','--','-.',':']
ils_sim = 0 #linestyle index (simulation)
ils_phi = 0 #linestyle index (dPhi)
ils_phicomb = 0 #3
ils_dB = 1  #linestyle index (dBpara)
ils_dU = 2  #linestyle index (dUperp)
clr_sim = 'k'
clr_phi = 'darkorange'
clr_phi_t_avg = 'r'
clr_phicomb = 'darkorange' #'r'
clr_dU = 'g'
clr_dB = 'm'
font_size = 9
#fontweight_legend = 'light' #'normal' #--doesn't work..
lbl_sim = r'$\mathrm{simulation}$'
lbl_phi = r'${\rm theory}$ ($\delta\Phi_{\rm tot}$)'
lbl_phicomb = r'${\rm theory}$ ($\delta\Phi_{\rm tot}$)'
#lbl_dU = r'${\rm theory}$ (${\rm only}$ $\delta\Phi_{\rm mhd}$)'
#lbl_dB = r'${\rm theory}$ (${\rm only}$ $\delta\Phi_{\rm kin}$)'
lbl_dU = r'${\rm theory}$ ($\delta\Phi_{\rm mhd}$)'
lbl_dB = r'${\rm theory}$ ($\delta\Phi_{\rm kin}$)'

In [ ]:
yr_max_f_w

In [ ]:
#--set figure real width
width = width_2columns
#
fig1 = plt.figure(figsize=(3,3))
fig1.set_figheight((np.sqrt(5.0)-1.0)/2.0 * width*1.46)#1.35)#1.5)
fig1.set_figwidth(width)
grid = plt.GridSpec(3, 2, hspace=0.08, wspace=0.05)
###--f vs w_perp - check
#
#ax1a = fig1.add_subplot(grid[0:1,1:2])
# beta = 0.3
ax2a = fig1.add_subplot(grid[0:1,0:1])
for sp in np.arange(nspecies):
    plt.plot(vprp[sp][0,:,0],f0_vprp[sp],':',linewidth=line_thick)
    plt.plot(vprp[sp][0,:,0],f_vprp[sp][-1],linewidth=line_thick)
#plt.plot(vprp,f_final,'k--',linewidth=line_thick)
#ax2a.fill_between(vprp,f_final,f_init,facecolor='grey',alpha=0.25)
plt.axvline(x=1.0,c='k',linestyle=':',linewidth=line_thick_aux,alpha=0.66)
plt.axvline(x=vA01,c='k',linestyle='--',linewidth=line_thick_aux,alpha=0.66)
plt.text(0.833*vA01,0.9*yr_max_f_w,r'$w_\perp = v_{\mathrm{A}0}$',va='top',ha='left',color='k',rotation=90,fontsize=font_size)
plt.xlim(xr_min_w,xr_max_w)
plt.ylim(yr_min_f_w,yr_max_f_w)
plt.xscale("log")
plt.ylabel(r'$\langle f(w_\perp)\rangle$',fontsize=font_size)
plt.title(r'$\beta_{\mathrm{i}0}=0.3$',fontsize=font_size)
ax2a.set_xticklabels('')
#ax2a.set_yticklabels('')
ax2a.tick_params(labelsize=font_size)
plt.text(1.067*xr_min_w,1.0,r'$\mathrm{(a)}$',va='center',ha='left',color='k',rotation=0,fontsize=font_size+1)#,weight='bold')

In [ ]:
np.mean(Dprpprp_t_avg[sp],axis=0).shape

In [ ]:
stdDprpprp_vprp[sp].shape

In [ ]:
ax2b = fig1.add_subplot(grid[1:2,0:1])
#plt.plot(vprp,Dprpprp,c=clr_sim,ls=lnstyl[ils_sim],linewidth=line_thick,label=lbl_sim)
#ax2b.fill_between(vprp,Dprpprp-c_pm_std*stdDprpprp_vprp,Dprpprp+c_pm_std*stdDprpprp_vprp,facecolor='grey',alpha=0.33)
for sp in np.arange(nspecies):
    plt.plot(vprp[sp][0,:,0],np.mean(Dprpprp_t_avg[sp],axis=0),c=clr_sim,ls=lnstyl[ils_sim],linewidth=line_thick,label=lbl_sim)
    #ax2b.fill_between
    plt.fill_between(vprp[sp][0,:,0],np.mean(Dprpprp_t_avg[sp],axis=0)-c_pm_std*stdDprpprp_vprp[sp],np.mean(Dprpprp_t_avg[sp],axis=0)+c_pm_std*stdDprpprp_vprp[sp],facecolor='grey',alpha=0.33)
    #plt.plot(vprp,Dprpprp_Phik,c=clr_phi,ls=lnstyl[ils_phi],linewidth=line_thick,label=lbl_phi)
    ##plt.plot(vprp,Dprpprp_Phik_comb,c=clr_phicomb,ls=lnstyl[ils_phicomb],linewidth=line_thick,label=lbl_phicomb)
    #plt.plot(vprp,Dprpprp_Uk,c=clr_dU,ls=lnstyl[ils_dU],linewidth=line_thick,label=lbl_dU)
    #plt.plot(vprp,Dprpprp_Bzk,c=clr_dB,ls=lnstyl[ils_dB],linewidth=line_thick,label=lbl_dB)
    plt.plot(vprp[sp][0,:,0],Dprpprp_Phik_t_avg[sp],c=clr_phi,ls=lnstyl[ils_phi],linewidth=line_thick,label=lbl_phi)
    #plt.plot(vprp,Dprpprp_Uk_t_avg,c=clr_dU,ls=lnstyl[ils_dU],linewidth=line_thick,label=lbl_dU)
    #plt.plot(vprp,Dprpprp_Bzk_t_avg,c=clr_dB,ls=lnstyl[ils_dB],linewidth=line_thick,label=lbl_dB)
plt.axvline(x=1.0,c='k',linestyle=':',linewidth=line_thick_aux,alpha=0.66)
plt.axvline(x=vA01,c='k',linestyle='--',linewidth=line_thick_aux,alpha=0.66)
plt.xlim(xr_min_w,xr_max_w)
plt.ylim(yr_min_D_w,yr_max_D_w)
plt.xscale("log")
plt.yscale("log")
plt.ylabel(r'$Q_\mathrm{inj}^{-1}v_{\mathrm{th,i}0}^{-2}\langle D_{\perp\perp}^E\rangle$',fontsize=font_size)
ax2b.set_xticklabels('')
#ax2b.set_yticklabels('')
ax2b.tick_params(labelsize=font_size)
#plt.legend(loc='best',markerscale=0.5,frameon=False,bbox_to_anchor=(0.55, 0.425),fontsize=font_size,ncol=1,handlelength=2.5)#,fontweight=fontweight_legend)
plt.legend(loc='best',markerscale=0.5,frameon=False,bbox_to_anchor=(0.6, 0.425),fontsize=font_size,ncol=1,handlelength=1.66)#,fontweight=fontweight_legend)
#plt.legend(loc='best',markerscale=0.5,frameon=False,bbox_to_anchor=(0.56, 0.41),fontsize=font_size,ncol=1,handlelength=2.5)#,fontweight=fontweight_legend)
#plt.text(1.067*xr_min_w,0.9*yr_max_D_w,r'$\mathrm{(b)}$',va='top',ha='left',color='k',rotation=0,fontsize=font_size+1)#,weight='bold')
plt.text(1.067*xr_min_w,0.9*yr_max_D_w,r'$\alpha_{\phi}$= '+'%.2f' %alphaPHI,va='top',ha='left',color='k',rotation=0,fontsize=font_size+4)#,weight='bold')
plt.text(1.067*xr_min_w,0.7*yr_max_D_w,r'$c0exp= $'+'%.3f' %c0exp,va='top',ha='left',color='k',rotation=0,fontsize=font_size+4)#,weight='bold')
#plt.savefig("../figures/"+name+"/SH_fit_scan/50-76/alp"+str(alphaPHI)+"c0exp"+str(c0exp)+".png")

In [ ]:
# Do a parameter scan over c0exp and alphaPHI, and save plots

In [ ]:
c0expscan = np.arange(0.03,0.132,0.002)
alphaPHIscan = np.arange(0.65,1.37,0.02)


c0exp = 0.05 #beta=0.3 0.05 #0.05
alphaPHI = 1.25#1.1 #1.25 #1.2
cst01 = 0.9 #0.9   #1.05
if cooling_corr:
    interv = interv+"coolcorr/"

#outsp = "p/"
#outsp = "he/"
outsp = "o5/"

for c0exp in c0expscan:
    for alphaPHI in alphaPHIscan:
        dPhi_v = [] #Phi vs vprp - different species may have different vprp scales
        dPhi_v_t = []
        Dprpprp_Phik = []
        Dprpprp_Phik_t = []
        Dprpprp_Phik_t_avg = []
        Qprp_Phik = []
        Qprp_Phik_t = []
        Qprp_Phik_t_avg = []
        normD_phi01 = []
        normD_phi01_t_avg = []
        Qprp_k_phik = []
        Qprp_k_phik_t_avg = []
        for sp in np.arange(nspecies):
            # calculate k phi/beta and interpolate and convert to v-space
            dPhi_k = np.sqrt(kprp_phi_rho*Phikprp/np.sqrt(betai0)) # attention! spectra were originally computed in kprp*di units!
            dPhi_v.append(np.interp(vprp[sp][0,:,0], charge[plotsp[sp]]/np.sqrt(mass[plotsp[sp]]) * alphaPHI/kprp_phi_rho[::-1], dPhi_k[::-1]))
            dPhi_v[sp] /= betai0 #thermal units
            #
            dPhi_k_t = np.zeros([len(kprp_phi_rho),it1-it0+1])
            dPhi_v_t.append(np.zeros([len(vprp[sp][0,:,0]),it1-it0+1]))
            for ind in range(it0,it1+1):
              dPhi_k_t[:,ind-it0] = np.sqrt(kprp_phi_rho[:]*Phikprp_t[:,ind-it0]/np.sqrt(betai0)) # attention! spectra were originally computed in kprp*di units!
              dPhi_v_t[sp][:,ind-it0] = dPhi_v_t[sp][:,ind-it0] = np.interp(vprp[sp][0,:,0], charge[plotsp[sp]]/np.sqrt(mass[plotsp[sp]]) * alphaPHI/kprp_phi_rho[::-1], dPhi_k_t[::-1,ind-it0])
              dPhi_v_t[sp][:,ind-it0] = dPhi_v_t[sp][:,ind-it0] / betai0
            #
            #--second: corresponding diffusion coefficients
            #
            # multiply the potential by the charge of the species of interest
            dPhi_v[sp] *= charge[plotsp[sp]]
            dPhi_v_t[sp] *= charge[plotsp[sp]]
            #
            Dprpprp_Phik.append((dPhi_v[sp]**3.) / (vprp[sp][0,:,0]**2.) )           #diffusion coefficient associated to Phi_tot (in w_perp/v_th)
            # We want dPhi_v - the fully combined phi
            # Dprpprp_Phik calculates using time-averaged phi
            #
            Dprpprp_Phik_t.append(np.zeros([len(vprp[sp][0,:,0]),it1-it0+1]) )
            for ind in range(it0,it1+1):
              Dprpprp_Phik_t[sp][:,ind-it0] = (dPhi_v_t[sp][:,ind-it0]**3.) / (vprp[sp][0,:,0]**2.)
              if exp_corr:
                 Dprpprp_Phik_t[sp][:,ind-it0] = Dprpprp_Phik_t[sp][:,ind-it0]*np.exp(-c0exp*(vprp[sp][0,:,0]**2./dPhi_v_t[sp][:,ind-it0]))
            Dprpprp_Phik_t[sp] *= charge[plotsp[sp]] / mass[plotsp[sp]]
            Dprpprp_Phik_t_avg.append(np.sum(Dprpprp_Phik_t[sp],axis=1)/float(it1-it0+1) )
            #
            # Dprpprp_Phik_t_avg calculates time dependent Dprpprp, and takes a time average
            if exp_corr:
               Dprpprp_Phik[sp] *= np.exp(-c0exp*(vprp[sp][0,:,0]**2./dPhi_v[sp]))           #apply exponential correction (but using: Phi_comb or Phi_tot)
            #

            Dprpprp_Phik[sp] *= charge[plotsp[sp]] / mass[plotsp[sp]]
            # Matt's boardwork missed a factor of m in the definition of D (Eq.7 in Silvio's paper)
            #--third: corresponding differential heating
            #
            Qprp_Phik.append(- Dprpprp_Phik[sp].reshape(1,vprp[sp].shape[1]) * dfdwprp[sp] )           #differential perpendicular heating associated to Phi_tot (in w_perp/v_th)
            #^ has time as first dimension instead of second
            Qprp_Phik_t.append(np.zeros([len(vprp[sp][0,:,0]),it1-it0+1]))
            for ind in range(it0,it1+1):
              Qprp_Phik_t[sp][:,ind-it0] = - Dprpprp_Phik_t[sp][:,ind-it0] * dfdwprp_t_interp[sp][ind-it0,:]
            Qprp_Phik_t_avg.append(np.sum(Qprp_Phik_t[sp],axis=1)/float(it1-it0+1))

            ### normalizations (made in w_perp/v_th space, then ported to k_perp*rho_th space)
            #
            #--Dprpprp vs wprp
            #
            #cst01 = 0.9 #1.0
            #
            i01 = np.where(vprp[sp][0,:,0] > 1.0 / (charge[plotsp[sp]]/np.sqrt(mass[plotsp[sp]]) /alphaPHI))[0][0]  #--match Dprpprp_phi with Dprpprp_sim (0.6?)  #1.0/alphaPHI
            #
            #normalisations to make things line up through a single w_perp - we don't care about the individual components

            if apply_smoothing:
              normD_phi01.append(cst01*Dprpprp[sp][:,i01] / Dprpprp_Phik[sp][i01] )#one normalisation for all t, or each t - plot this vs t to see if it changes much in time, taking the mean for now
              normD_phi01_t_avg.append(cst01*Dprpprp[sp][:,i01] / Dprpprp_Phik_t_avg[sp][i01] )
            else:
              normD_phi01 = cst01*Dprpprp_smooth[sp][:,i01] / Dprpprp_Phik[sp][i01]
              normD_phi01_t_avg = cst01*Dprpprp_smooth[sp][:,i01] / Dprpprp_Phik_t_avg[sp][i01]

            #
            Dprpprp_Phik[sp] *= np.mean(normD_phi01[sp])
            Dprpprp_Phik_t_avg[sp] *= np.mean(normD_phi01_t_avg[sp])
            #
            print('\n')
            print('### betai0 = 0.3 ###')
            print('species',sp)
            print('* kappa0 values: ')
            print('  effective kappa0_PHI = ',alphaPHI)
            print('* D_perpperp normalizations: ')
            print('  normD_Phi =',np.mean(normD_phi01[sp]))
            print('  normD_Phi_t_avg =',np.mean(normD_phi01_t_avg[sp]))
            #
            #--Qprp vs wprp (apply same normalization as for Dprpprp)
            Qprp_Phik[sp] *= np.mean(normD_phi01[sp])
            Qprp_Phik_t_avg[sp] *= np.mean(normD_phi01_t_avg[sp])
            #
            #####
            #
            # << vs k_perp >>
            #
            # -> just interpolate dQperp/dwperp into k_perp*rho_th
            #    (plus extra contribution due to the fact that we
            #     need to compute dQperp/dlog(k_perp) and not dQperp/dk_perp)
            #    [ normalization should already be consistent! ]
            #
            #--heating in k_perp from Phi spectrum
            Qprp_k_phik.append(np.zeros((Qprp_Phik[sp].shape[0],kprp_phi_rho.shape[0])))

            for t in np.arange(Qprp_Phik[sp].shape[0]):
                Qprp_k_phik[sp][t] = np.interp( kprp_phi_rho, alphaPHI/vprp[sp][0,::-1,0], Qprp_Phik[sp][t,::-1] )
            Qprp_k_phik[sp] *= (alphaPHI/kprp_phi_rho.reshape((1,kprp_phi_rho.shape[0])))

            Qprp_k_phik_t_avg.append(np.interp( kprp_phi_rho, alphaPHI/vprp[sp][0,::-1,0], Qprp_Phik_t_avg[sp][::-1] ))
            Qprp_k_phik_t_avg[sp] *= (alphaPHI/kprp_phi_rho)
            Qprp_k_phik_t_avg[sp] /= np.log(10.)

        xr_max_w = []
        yr_max_f_w = []
        yr_max_Q_w = []
        for sp in np.arange(nspecies):
            xr_max_w.append(np.max(vprp[sp]) )
            yr_max_f_w.append(1.025*np.max([np.max(f_vprp[sp]),np.max(f0_vprp[sp])]) )
            yr_max_Q_w.append(1.1*np.max([np.max(Qprp_vprp[sp])]))
        xr_max_w = np.max(np.array(xr_max_w))#[plotsp]))
        yr_max_f_w = np.max(np.array(yr_max_f_w))#[plotsp]))
        yr_max_Q_w = np.max(np.array(xr_max_w))#[plotsp]))

        fig2 = plt.figure()
        ax3b = fig2.add_subplot(111)
        #plt.plot(vprp,Dprpprp,c=clr_sim,ls=lnstyl[ils_sim],linewidth=line_thick,label=lbl_sim)
        #ax2b.fill_between(vprp,Dprpprp-c_pm_std*stdDprpprp_vprp,Dprpprp+c_pm_std*stdDprpprp_vprp,facecolor='grey',alpha=0.33)
        for sp in np.arange(nspecies):
            plt.plot(vprp[sp][0,:,0],np.mean(Dprpprp_t_avg[sp],axis=0),c=clr_sim,ls=lnstyl[ils_sim],linewidth=line_thick,label=lbl_sim)
            #ax2b.fill_between
            plt.fill_between(vprp[sp][0,:,0],np.mean(Dprpprp_t_avg[sp],axis=0)-c_pm_std*stdDprpprp_vprp[sp],np.mean(Dprpprp_t_avg[sp],axis=0)+c_pm_std*stdDprpprp_vprp[sp],facecolor='grey',alpha=0.33)
            #plt.plot(vprp,Dprpprp_Phik,c=clr_phi,ls=lnstyl[ils_phi],linewidth=line_thick,label=lbl_phi)
            ##plt.plot(vprp,Dprpprp_Phik_comb,c=clr_phicomb,ls=lnstyl[ils_phicomb],linewidth=line_thick,label=lbl_phicomb)
            #plt.plot(vprp,Dprpprp_Uk,c=clr_dU,ls=lnstyl[ils_dU],linewidth=line_thick,label=lbl_dU)
            #plt.plot(vprp,Dprpprp_Bzk,c=clr_dB,ls=lnstyl[ils_dB],linewidth=line_thick,label=lbl_dB)
            plt.plot(vprp[sp][0,:,0],Dprpprp_Phik_t_avg[sp],c=clr_phi,ls=lnstyl[ils_phi],linewidth=line_thick,label=lbl_phi)
            #plt.plot(vprp,Dprpprp_Uk_t_avg,c=clr_dU,ls=lnstyl[ils_dU],linewidth=line_thick,label=lbl_dU)
            #plt.plot(vprp,Dprpprp_Bzk_t_avg,c=clr_dB,ls=lnstyl[ils_dB],linewidth=line_thick,label=lbl_dB)
        plt.axvline(x=1.0,c='k',linestyle=':',linewidth=line_thick_aux,alpha=0.66)
        plt.axvline(x=vA01,c='k',linestyle='--',linewidth=line_thick_aux,alpha=0.66)
        plt.xlim(xr_min_w,xr_max_w)
        plt.ylim(yr_min_D_w,yr_max_D_w)
        plt.xscale("log")
        plt.yscale("log")
        plt.ylabel(r'$Q_\mathrm{inj}^{-1}v_{\mathrm{th,i}0}^{-2}\langle D_{\perp\perp}^E\rangle$',fontsize=font_size)
        ax3b.set_xticklabels('')
        #ax2b.set_yticklabels('')
        ax3b.tick_params(labelsize=font_size)
        #plt.legend(loc='best',markerscale=0.5,frameon=False,bbox_to_anchor=(0.55, 0.425),fontsize=font_size,ncol=1,handlelength=2.5)#,fontweight=fontweight_legend)
        plt.legend(loc='best',markerscale=0.5,frameon=False,bbox_to_anchor=(0.6, 0.425),fontsize=font_size,ncol=1,handlelength=1.66)#,fontweight=fontweight_legend)
        #plt.legend(loc='best',markerscale=0.5,frameon=False,bbox_to_anchor=(0.56, 0.41),fontsize=font_size,ncol=1,handlelength=2.5)#,fontweight=fontweight_legend)
        plt.text(1.067*xr_min_w,0.9*yr_max_D_w,r'$\alpha_{\phi}$= '+'%.2f' %alphaPHI,va='top',ha='left',color='k',rotation=0,fontsize=font_size+4)#,weight='bold')
        plt.text(1.067*xr_min_w,0.7*yr_max_D_w,r'$c0exp= $'+'%.3f' %c0exp,va='top',ha='left',color='k',rotation=0,fontsize=font_size+4)#,weight='bold')
        print("../figures/"+name+"/SH_fit_scan/"+outsp+interv+"alp"+'%.2f' %alphaPHI+"c0exp"+'%.3f' %c0exp+".png")
        plt.savefig("../figures/"+name+"/SH_fit_scan/"+outsp+interv+"alp"+'%.2f' %alphaPHI+"c0exp"+'%.3f' %c0exp+".png")
        plt.close()

In [ ]:
str(alphaPHI)

In [ ]:
c0expscan

In [ ]:
alphaPHIscan

In [ ]:
str(0.045)

In [ ]:
'%.2f' %c0expscan[2]+"hello"

In [ ]:
plt.plot(vprp[sp][0,:,0],Dprpprp_Phik_t_avg[sp],c=clr_phi,ls=lnstyl[ils_phi],linewidth=line_thick,label=lbl_phi)
plt.ylim(yr_min_D_w,yr_max_D_w)
plt.xlim(xr_min_w,xr_max_w)
plt.xscale("log")
plt.yscale("log")

In [ ]:
plt.fill_between(vprp[sp][0,:,0],np.mean(Dprpprp_t_avg[sp],axis=0)-c_pm_std*stdDprpprp_vprp[sp],np.mean(Dprpprp_t_avg[sp],axis=0)+c_pm_std*stdDprpprp_vprp[sp],facecolor='grey',alpha=0.33)
plt.xlim(xr_min_w,xr_max_w)
plt.xscale("log")
plt.yscale("log")

In [ ]:
Dprpprp_Phik_t_avg[sp].shape

In [ ]:
Dprpprp_Phik_t_avg.append(np.sum(Dprpprp_Phik_t,axis=1)/float(it1-it0+1) )

In [ ]:
(np.sum(Dprpprp_Phik_t[sp],axis=1)/float(it1-it0+1)).shape

In [ ]:
Qprp_Phik_t[sp].shape

In [ ]:
Qprp_Phik[sp]

In [ ]:
vprp[sp][0,:,0].shape

In [ ]:
dwdfwprp_t_interp[sp]

In [ ]:
cst01*Dprpprp[sp][:,i01] / Dprpprp_Phik[i01]

In [ ]:
i01

In [ ]:
len(Dprpprp_Phik[sp])

In [ ]:
Dprpprp_Phik_t_avg[sp].shape

In [ ]:
np.interp( kprp_phi_rho, alphaPHI/vprp[0][0,::-1,0], Qprp_Phik_t_avg[0][::-1] )

In [ ]:
kprp_phi_rho.shape

In [ ]:
(alphaPHI/vprp[0][0,::-1,0]).shape

In [ ]:
(Qprp_Phik_t_avg[0][::-1]).shape

In [ ]:
Qprp_Phik_t[sp].shape

In [ ]:
np.sum(Qprp_Phik_t[sp],axis=1).shape

In [ ]:
Qprp_Phik_t_avg[sp].shape

In [ ]:
(np.sum(Qprp_Phik_t,axis=1)/float(it1-it0+1)).shape

In [ ]:
a=np.arange(10)

In [ ]:
a

In [ ]:
np.std(a)

In [ ]:
a

In [ ]:
b = np.arange(9)

In [ ]:
b

In [ ]:
del a,b

In [ ]:
a

In [ ]:
np.arange(1,5)